In [4]:
import torch
import resnet1d as r
from data import *
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
from tqdm import tqdm


def model_pipeline():


    #make the model, data and optimization problem
    model, criterion, optimizer, trainloader, testloader, validationloader = create()

    #train the model
    train(model, trainloader, criterion, optimizer, validationloader)

    #test the model
    print("Accuracy test: ",test(model, testloader))
        
    #return model

def create():
    resnet1d = r.CreateResNet1D(num_classes=35)
    #Create a model
    model = resnet1d.to(device)
    
    #Create the loss and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    trainloader,testloader,validationloader = getData()

    return model, criterion, optimizer,trainloader, testloader, validationloader

# Function to train a model.
def train(model, trainloader, criterion, optimizer, validationloader):
 

    model.train()
    losses, valacc = [], []  

    for epoch in range(5):
        
        progress_bar = tqdm(trainloader, desc=f'Training epoch {epoch}', leave=False)
        
        for batch, (images, labels) in enumerate(progress_bar):
        
            loss = train_batch(images, labels,model, optimizer, criterion)
            progress_bar.update(1)
            
            logs = {"loss": loss.detach().item()}
            progress_bar.set_postfix(**logs)
        
            losses.append(loss.item())

    return np.mean(losses)

def train_batch(images, labels, model, optimizer, criterion):

    #insert data into cuda if available
    images,labels = images.to(device), labels.to(device)
    
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    #backward pass
    loss.backward()

    #step with optimizer
    optimizer.step()

    return loss

def test(model, test_loader):
    model.eval()

    with torch.no_grad():
        correct, total = 0, 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            oututs = model(images)
            _, predicated = torch.max(oututs.data, 1)
            total += labels.size(0)

            correct += (predicated == labels).sum().item()

        return correct/total

In [6]:
model_pipeline()

RuntimeError: Given groups=1, weight of size [64, 3, 7], expected input[32, 1, 16000] to have 3 channels, but got 1 channels instead